In [1]:
# Import các thư viện cần thiết
import numpy as np 
import torch 
from torch.nn import Parameter 
from torch.nn.modules.module import  Module 
import torch.nn.functional as F
import math 
from torch import nn 

In [2]:
# Xây dựng BaseModel 
class PruningModel(Module):
	def prune_by_std(self, s=0.25):
		"""
			Cắt tỉa trọng số sử dụng độ lệch chuẩn (std - 
			standard deviation) được gọi là ngưỡng (threshold)
		"""
		# Lưu ý rằng thuật ngữ `module`` ở đây được hiểu là các 
		# `lớp (layer)`, ví dụ: fc1, fc2, fc3
		for name, module in self.named_modules():
			if name in ['fc1', 'fc2', 'fc3']:
				threshold = np.std(module.weight.data.cpu().numpy()) * s
				print(f">>> Cắt tỉa với ngưỡng: {threshold} cho layer {name}")
				module.prune(threshold)

In [ ]:
# Xây dựng module cắt tỉa 
class MaskedLinear(Module):
	def __init__(self, in_features, out_features, bias=True):
		super(MaskedLinear, self).__init__()
		self.in_features = in_features
		self.out_features = out_features
		self.weight = Parameter(torch.Tensor(out_features, in_features))
		# Khởi tạo ma trận toàn số 1
		self.mask = Parameter(
			torch.ones([out_features, in_features]), 
			requires_grad=False
		)
		if bias: 
			self.bias = Parameter(torch.Tensor(out_features))
		else:
			self.register_parameter('bias', None)
		self.reset_parameters() 

	def reset_parameters(self):
		stdv = 1. / math.sqrt(self.weight.size(1))
		# Phân phối đều trọng số mô hình trong khoảng [-stdv, stdv]
		self.weight.data.uniform_(-stdv, stdv)
		if self.bias is not None:
			self.bias.data.uniform_(-stdv, stdv)

	def forward(self, input):
		return F.linear(input, self.weight * self.mask, self.bias)

	def __repr__(self):
		return self.__class__.__name__ + '(' \
			+ 'in_features=' + str(self.in_features) \
			+ ', out_features=' + str(self.out_features) \
			+ ', bias=' + str(self.bias is not None) + ')'

	def prune(self, threshold):
		"""Hàm tuỳ chỉnh cắt tỉa (prune) với mặt nạ (mask)"""
		weight_device = self.weight.device
		mask_device = self.mask.device 
		# Đưa tensor từ GPU về CPU và chuyển tensor về mảng numpy
		tensor = self.weight.data.cpu().numpy()
		mask = self.mask.data.cpu().numpy()
		# Sau khi cắt tỉa những weight nào không cần nữa thì sẽ thành số 0
		new_mask = np.where(abs(tensor) < threshold, 0, mask)
		# Apply trọng số và mặt nạ mới
		self.weight.data = torch.from_numpy(tensor * new_mask).to(weight_device)
		self.mask.data = torch.from_numpy(new_mask).to(mask_device)